# Tabular Data Formats: Tidying Up

Check the Python version and load miscellaneous utility functions and classes.

In [ ]:
import sys
sys.version

In [ ]:
from src.setup import *

## Types of Grime

There are roughly two families of problems we find in data sets.  Not every problem neatly divides into these families, or at least it is not always evident which side something falls on without knowing the root cause.  But in a general way we can think of structural problems in the formatting of data versus content problems in the actual values recorded.  

On the structural branch a format used to encode a data set might simply "put values in the wrong place" in one way or another.  On the content side, the data format itself is correct, but implausible or wrong values have snuck in via flawed instruments, transcription errors, numeric overflows, or through other pitfalls of the recording process.

In the case of structural problems, we almost always need manual remediation of the data.  Exactly where the bytes that make up the data go wrong can vary enormously, and usually does not follow a pattern that lends itself to a single high-level description.  Often we have a somewhat easier time with the content problems, but at the same time they are more likely to be irremediable even with manual work.  Consider this small comma-separated value (CSV) data source, describing a 6th grade class:

```
Student#,Last Name,First Name,Favorite Color,Age
1,Johnson,Mia,periwinkle,12
2,Lopez,Liam,blue,green,13
3,Lee,Isabella,,11
4,Fisher,Mason,gray,-1
5,Gupta,Olivia,9,102
6,,Robinson,,Sophia,,blue,,12
```

In a friendly way, we have a header line that indicates reasonable field names and provides a hint as to the meaning of each column.  Programmatically, we may not wish to work with the punctuation marks and spaces inside some field names, but that is a matter of tool convenience that we can address with the APIs (*application programming interfaces*; the functions and methods of a library) that data processing tools give us (perhaps by renaming them).

Let us think about each record in turn.  
* Mia Johnson, student 1, seems to have a problem-free record.  Her row has five values separated by four commas, and each data value meets our intuitive expectations about the data type and value domain.  The problems start hereafter.

* Liam Lopez has too many fields in his row.  However, both columns 4 and 5 seem clearly to be in the lexicon of color names.  Perhaps a duplicate entry occurred or the compound color "blue-green" was intended.  Structurally the row has issues, but several plausible remediations suggest themselves.

* Isabella Lee is perhaps no problem at all.  One of her fields is empty, meaning no favorite color is available.  But structurally, this row is perfectly fine for CSV format.  We will need to use some domain or problem knowledge to decide how to handle the missing value.

* Mason Fisher is perhaps similar to Isabella.  The recorded age of -1 makes no sense in the nature of "age" as a data field, at least as we usually understand it (but maybe the encoding intends something different).  On the other hand, -1 is one of several placeholder values used very commonly to represent missing data.  We need to know our specific problem to know whether we can process the data with a missing age, but many times we can handle that.  However, we still need to be careful not to treat the -1 as a plain value; for example, the mean, minimum, or standard deviation of ages might be thrown off by that.

* Olivia Gupta starts to present a trickier problem.  Structurally her row looks perfect.  But '9' is probably not a string in our lexicon of color names.  And under our understanding of the data concerning a 6th grade class, we don't expect 102 year old students to be in it.  To solve this row, we really need to know more about the collection procedure and the intention of the data.  Perhaps a separate mapping of numbers to colors exists somewhere.  Perhaps an age of 12 was mistranscribed as 102; but also perhaps a 102 year old serves as a teaching assistant in this class and not only students are recorded.

* Sophia Robinson returns us to what looks like an obvious structural error.  The row, upon visual inspection, contains perfectly good and plausible values, but they are separated by duplicate commas.  Somehow, persumably, a mechanical error resulted in the line being formatted wrongly.  However, most high-level tools are likely to choke on the row in an uninformative way, and we will probably need to remediate the issue more manually.

We have a pretty good idea what to do with these six rows of data, and even re-entering them from scratch would not be difficult.  If we had a million rows instead, the difficulty would grow greatly, and would require considerable effort before we arrived at usable data.

## Nomenclature

In this course I will use the terms *feature*, *field*, *measurement*, *column*, and occasionally *variable* more-or-less interchangeably.  Likewise, the terms *row*, *record*, *observation*, and *sample* are also near synonyms.  *Tuple* is used for the same concept when discussing databases (especially academically). In different academic or business fields, different ones of these terms are more prominent; and likewise different software tools choose among these. 

Conceptually, most data can be thought of as a number of occasions on which we measure various attributes of a common underlying *thing*.  In most tools, it is usually convenient to put these observations/samples each in a row; and correspondingly to store each of the measurements/features/fields pertaining to that thing in a column containing corresponding data for other comparable *things*.

## Tidy Data

Tidy data carefully separates variables (the columns of a table; also called features or fields) from observations (the rows of a table; also called samples).  At the intersection of these two, we find values, one data item (datum) in each cell.  Unfortunately, the data we encounter is often not arranged in this useful way, and it requires *normalization*.  In particular, what are really values are often represented either as columns or as rows instead.

In [ ]:
students = pd.read_csv('data/students-scores.csv')
students

This view of the data is easy for humans to read.  We can see trends in the scores each student received over several years of education.  Moreover, this format might lend itself to useful visualizations fairly easily.

In [ ]:
# Generic conversion of letter grades to numbers
def num_score(x):
    to_num = {'A+': 4.3, 'A': 4, 'A-': 3.7,
              'B+': 3.3, 'B': 3, 'B-': 2.7,
              'C+': 2.3, 'C': 2, 'C-': 1.7}
    return x.map(lambda x: to_num.get(x, x))

This next cell uses a "fluent" programming style that may look unfamiliar to some Python programmers.  I discuss this style in the section below on data frames.  The fluent style is used in many data science tools and languages. For example, this is typical Pandas code that plots the students' scores by year.

In [ ]:
(students
     .set_index('Last Name')
     .drop('First Name', axis=1)
     .apply(num_score)
     .T
     .plot(title="Student score by year")
     .legend(bbox_to_anchor=(1, .75))
)
plt;

<font color="darkcyan">**Question for students**: Should we review how that Pandas command was constructed?</font>

This data layout exposes its limitations once the class advances to 7th grade, or if we were to obtain 3rd grade information.  To accommodate such additional data, we would need to change the number and position of columns, not simply add additional rows.  It is natural to make new observations or identify new samples (rows), but usually awkward to change the underlying variables (columns).

The particular class level (e.g. 4th grade) that a letter grade pertains to is, at heart, a value not a variable.  Another way to think of this is in terms of independent variables versus dependent variables.  Or in machine learning terms, features versus target.  In some way, the class level might correlate with or influence the resulting letter grade; perhaps the teachers at the different levels have different biases, or children of a certain age lose or gain interest in schoolwork, for example.

For most analytic purposes, this data would be more useful if we make it tidy (normalized) before further processing.  In Pandas, the `DataFrame.melt()` method can perform this tidying.  We pin some of the columns as `id_vars`, and we set a name for the combined columns as a variable and the letter grade as a single new column.  This Pandas method is slightly magical, and takes some practice to get used to.  The key thing is that it preserves data, simply moving it between column labels and data values.

In [ ]:
students.melt(
    id_vars=["Last Name", "First Name"], 
    var_name="Level",
    value_name="Score"
).set_index(['First Name', 'Last Name', 'Level'])

Within the Tidyverse, specifically within the **tidyr** package, there is a function `pivot_longer()` that is similar to Pandas' `.melt()`.  The aggregation names and values have parameters spelled `names_to` and `values_to`, but the operation is the same.

The simple example above gives you a first feel for tidying tabular data.  To reverse the tidying operation that moves variables (columns) to values (rows), the `pivot_wider()` function in tidyr can be used.  In Pandas there are several related methods on DataFrames, including `.pivot()`, `.pivot_table()`, and `.groupby()` combined with `.unstack()`, which can create columns from rows (and do many other things too).

Having looked at the idea of tidyness as a general goal for tabular, let us being looking at specific data formats, beginning with comma-separated values and fixed-width files.